In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path("data/processing_checkpoint")
path_clean_dataset = DATA_DIR / "01_cleaned_train.parquet"

In [3]:
df_clean_churn = pd.read_parquet(path_clean_dataset)

In [4]:
df_clean_churn.sample(5)

,gender,level,userId,page,sessionId,itemInSession,length,song,artist,time,registration,metropolitan_area,region,operating_system,browser
2788134,M,free,1824100,NextSong,102844,1,166.34730,El Toro Pesado,Voces Del Rancho,2018-10-19 21:11:22,2018-09-21 11:52:34,Orlando-Kissimmee-Sanford,FL,Macintosh,Chrome
13067529,F,free,1415504,NextSong,169050,12,186.51383,Proud Mary,Creedence Clearwater Revival,2018-11-11 16:22:47,2018-11-07 19:57:03,Albany-Schenectady-Troy,NY,Macintosh,Safari
13084536,F,free,1335917,Roll Advert,185606,42,0.00000,No song,No artist,2018-11-15 22:13:31,2018-09-14 07:28:55,Eugene,OR,Macintosh,Chrome
4930416,M,free,1882952,NextSong,186388,49,205.08689,Les derniers aventuriers,La Rue Ketanou,2018-11-13 15:44:08,2018-09-01 09:05:21,Dallas-Fort Worth-Arlington,TX,Windows,Chrome
8553774,M,paid,1269254,NextSong,158956,64,202.37016,Cross The Tracks (We Better Go Back),Maceo And The Macks,2018-11-05 18:06:46,2018-09-29 01:18:12,Detroit-Warren-Dearborn,MI,Windows,Chrome


In [5]:
df_users_base = df_clean_churn[["userId", "gender", "registration", "operating_system", "browser"]].drop_duplicates(subset=["userId"])

In [6]:
len(df_users_base)

19140

In [7]:
df_users_base = df_users_base.set_index("userId")

In [8]:
df_unique_artists = (
    df_clean_churn.groupby("userId")["artist"].nunique().reset_index().rename(columns={"artist": "num_unique_artists"})
)
df_unique_artists = df_unique_artists.set_index("userId")
df_users_base = df_users_base.join(df_unique_artists)

In [9]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists
userId,,,,,
1754301,F,2018-07-09 19:26:57,Windows,Chrome,345
1240802,F,2018-09-06 20:21:56,Macintosh,Chrome,134
1664371,F,2018-09-18 14:30:39,Macintosh,Chrome,369
1372150,M,2018-04-24 05:00:51,Windows,Chrome,306
1767033,M,2018-07-21 07:17:47,iPad,Safari,4


In [10]:
df_page_counts = (
    df_clean_churn.groupby("userId")["page"]
    .value_counts()
    .unstack(fill_value=0)
)

df_page_counts.columns = [f"count_{col.replace(' ', '_').lower()}" for col in df_page_counts.columns]

In [11]:
df_users_base = df_users_base.join(df_page_counts)
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_logout,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade
userId,,,,,,,,,,,,,,,,,,,,,
1716813,F,2018-09-13 06:09:29,Macintosh,Chrome,103,0,0,1,1,1,...,3,106,0,0,1,0,0,0,5,0
1218107,M,2018-07-22 19:06:33,Macintosh,Chrome,55,0,0,1,1,1,...,0,57,3,1,3,0,0,1,0,1
1793165,M,2018-09-22 06:26:19,Windows,Chrome,321,1,10,15,0,0,...,8,372,26,1,4,0,0,14,14,2
1158332,F,2018-05-05 22:14:15,iPad,Safari,77,0,2,2,0,0,...,1,81,6,2,5,0,0,0,3,0
1818014,M,2018-10-17 09:53:17,Windows,Chrome,660,1,11,27,1,1,...,11,815,71,2,9,0,1,9,39,4


In [12]:
df_session_count = (
    df_clean_churn.groupby("userId")["sessionId"]
    .nunique()
    .reset_index(name="count_total_sessions")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_session_count)


In [13]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_nextsong,count_roll_advert,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions
userId,,,,,,,,,,,,,,,,,,,,,
1132977,M,2018-07-30 18:55:55,Windows,Firefox,242,1,5,12,0,0,...,258,32,0,3,0,0,1,9,2,7
1376643,F,2018-09-18 06:03:47,Macintosh,Chrome,149,0,8,7,0,0,...,155,9,0,2,0,0,1,5,1,4
1757863,M,2018-09-12 06:07:56,Macintosh,Firefox,112,0,3,3,1,1,...,118,10,0,1,0,0,5,10,0,5
1439898,M,2018-08-26 08:28:53,Macintosh,Chrome,219,0,0,9,0,0,...,234,15,1,1,0,0,1,10,0,8
1056011,F,2018-08-27 07:01:08,Windows,Chrome,77,0,2,3,0,0,...,78,6,1,2,0,0,0,3,0,3


In [14]:
# Build user lifecycle

df_last_time = (
    df_clean_churn.groupby("userId")["time"]
    .max()
    .reset_index(name="last_time")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_last_time)
df_users_base["user_lifecycle_h"] = df_users_base["last_time"] - df_users_base["registration"]
df_users_base["user_lifecycle_h"] = df_users_base["user_lifecycle_h"].dt.total_seconds() / 3600

df_users_base.head()

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_save_settings,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,2,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,2,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,1,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,2,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889


In [15]:
df_length = (
    df_clean_churn.groupby("userId")["length"].sum()
)
df_length = (
    df_length
    .reset_index(name="ttl_length")
    .set_index("userId")
)
df_users_base = df_users_base.join(df_length)
df_users_base.head(5)


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_settings,count_submit_downgrade,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,7,0,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,2,0,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,12,1,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,11,2,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,9,0,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168


In [16]:
df_item_per_session = (
    df_clean_churn.groupby("userId")["itemInSession"].max() / df_users_base["count_total_sessions"]
)

df_item_per_session = (
    df_item_per_session
    .reset_index(name="item_per_session")
    .set_index("userId")
)

df_users_base = df_users_base.join(df_item_per_session)


In [17]:
df_users_base["frequency"] = df_users_base["count_total_sessions"] / df_users_base["user_lifecycle_h"]

In [18]:
df_users_base.sample(5)

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1611454,F,2018-09-07 07:04:01,Windows,Firefox,118,0,4,4,0,0,...,0,3,8,0,3,2018-11-17 09:30:13,1706.436667,30764.76991,32.666667,0.001758
1971357,M,2018-09-20 08:22:25,iPhone,Safari,459,0,10,18,0,0,...,0,7,29,1,6,2018-10-29 09:35:52,937.224167,135201.16220,57.500000,0.006402
1599451,F,2018-08-26 19:51:25,Windows,Firefox,88,0,3,4,0,0,...,0,2,10,0,2,2018-11-01 16:37:22,1604.765833,22560.71672,41.000000,0.001246
1986148,F,2018-08-21 22:47:34,Windows,Firefox,723,1,18,23,0,0,...,1,14,59,10,25,2018-11-19 19:06:18,2156.312222,236737.71314,4.960000,0.011594
1848842,M,2018-08-18 11:30:49,Macintosh,Safari,847,1,16,27,1,1,...,1,12,55,1,14,2018-11-07 21:51:58,1954.352500,270488.44533,23.857143,0.007163


In [19]:
### df_users_base = df_users_base.drop(columns="user_lifecycle_h", errors="ignore")


In [20]:
"user_lifecycle_h" in df_users_base.columns

True

In [21]:
print(df_users_base["count_cancellation_confirmation"].unique())
df_users_base.rename(columns={"count_cancellation_confirmation":"churn_label"})

[1 0]


,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,churn_label,...,count_submit_upgrade,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency
userId,,,,,,,,,,,,,,,,,,,,,
1749042,M,2018-08-08 13:22:21,Windows,Chrome,797,5,18,33,1,1,...,0,9,51,0,9,2018-10-21 01:16:24,1763.900833,256456.28661,51.444444,0.005102
1563081,F,2018-09-21 03:25:18,Macintosh,Chrome,437,1,5,14,0,0,...,0,5,25,0,6,2018-11-03 02:00:13,1030.581944,123645.65861,29.666667,0.005822
1697168,F,2018-09-08 13:48:25,Macintosh,Firefox,1117,1,22,44,0,0,...,1,16,83,7,34,2018-11-19 23:59:55,1738.191667,386582.82117,12.941176,0.019561
1222580,M,2018-08-16 02:31:00,Macintosh,Safari,1252,8,35,50,1,1,...,2,17,85,5,20,2018-10-30 23:17:30,1820.775000,452234.08024,25.650000,0.010984
1714398,F,2018-09-19 18:23:35,Windows,Chrome,1102,0,22,51,0,0,...,0,19,79,0,14,2018-11-19 05:16:55,1450.888889,384934.72168,24.857143,0.009649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494594,M,2018-08-18 18:07:54,iPhone,Safari,12,0,0,0,0,0,...,0,0,0,0,1,2018-11-19 19:52:00,2233.735000,3047.43992,18.000000,0.000448
1036641,F,2018-07-09 12:04:12,Windows,Chrome,71,0,3,1,0,0,...,0,0,12,3,1,2018-11-19 23:58:21,3203.902500,17586.59262,102.000000,0.000312
1110980,F,2018-08-31 14:17:48,Windows,Chrome,51,0,0,4,0,0,...,0,0,5,0,1,2018-11-19 23:55:01,1929.620278,14253.34974,70.000000,0.000518


In [22]:
df_users_base["avg_songs_session"] = df_users_base["count_nextsong"] / df_users_base["count_total_sessions"]

In [26]:
mask = df_users_base["user_lifecycle_h"] < 24
df_users_base[mask]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.432500,15332.72528,79.0,0.095854,64.0
1444644,F,2018-10-06 04:01:43,Macintosh,Safari,40,0,0,2,0,0,...,0,0,0,1,2018-10-07 02:00:54,21.986389,9873.95455,46.0,0.045483,39.0
1039566,M,2018-10-07 10:56:31,iPhone,Safari,73,0,0,1,0,0,...,2,5,0,1,2018-10-07 16:32:59,5.607778,20749.91474,104.0,0.178324,75.0
1594905,M,2018-10-09 02:53:40,Windows,Chrome,304,2,2,11,0,0,...,4,15,0,1,2018-10-10 02:30:24,23.612222,84829.05169,399.0,0.042351,338.0
1745396,F,2018-11-07 15:38:32,Macintosh,Firefox,24,0,0,1,0,0,...,0,1,0,1,2018-11-07 17:02:16,1.395556,5221.47530,33.0,0.716561,23.0
1822231,M,2018-11-11 16:22:57,Windows,Firefox,11,0,1,1,0,0,...,0,1,0,1,2018-11-11 16:57:05,0.568889,2193.78487,28.0,1.757812,10.0
1792581,F,2018-11-12 12:06:34,Macintosh,Safari,17,0,0,1,0,0,...,0,0,0,1,2018-11-12 13:11:23,1.080278,4095.83557,38.0,0.925688,17.0
1094942,M,2018-11-13 18:48:19,Macintosh,Safari,140,1,0,4,0,0,...,0,7,0,1,2018-11-14 05:19:34,10.520833,38257.17415,193.0,0.095050,147.0
1351427,F,2018-11-14 10:14:32,Windows,Chrome,30,0,0,0,0,0,...,0,0,1,1,2018-11-14 12:37:59,2.390833,8783.92155,50.0,0.418264,29.0


In [39]:
df_users_base.loc[[1714714]]

,gender,registration,operating_system,browser,num_unique_artists,count_about,count_add_friend,count_add_to_playlist,count_cancel,count_cancellation_confirmation,...,count_thumbs_down,count_thumbs_up,count_upgrade,count_total_sessions,last_time,user_lifecycle_h,ttl_length,item_per_session,frequency,avg_songs_session
userId,,,,,,,,,,,,,,,,,,,,,
1714714,F,2018-09-30 22:59:16,Macintosh,Safari,65,1,0,0,1,1,...,0,2,0,1,2018-10-01 09:25:13,10.4325,15332.72528,79.0,0.095854,64.0
